## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

## 데이터 불러오기

In [ ]:
Fold = 5

# k-fold 데이터
for i in range(Fold):
    
    j = i+1
    path1 = './SavedData/Training_Fold%d'%j
    path2 = './SavedData/Validation_Fold%d'%j
    c1 = 'Training_%d = np.array(pd.read_csv(path1, sep = ",", header = None))'%j
    c2 = 'Validation_%d = np.array(pd.read_csv(path2, sep = ",", header = None))'%j
    exec(c1)
    exec(c2)

# 전체 학습용 데이터
Training_TotalData = np.array(pd.read_csv('./SavedData/Training_TotalData', sep = ",", header = None))
Training_TotalLabel = np.array(pd.read_csv('./SavedData/Training_TotalLabel', sep = ",", header = None))

## 데이터 라벨링

In [ ]:
DataNo = 100

Training_Label_size = int((DataNo - DataNo/Fold))
Validation_Label_size = int((DataNo/Fold))

print(Training_Label_size)
print(Validation_Label_size)


In [ ]:
Training_Label_knn = np.zeros(shape=2*Training_Label_size)
Validation_Label_knn = np.zeros(shape=2*Validation_Label_size)

# Training Label
Training_Label_knn[:Training_Label_size] = 0
Training_Label_knn[Training_Label_size:] = 1

# Validation Label
Validation_Label_knn[:Validation_Label_size] = 0
Validation_Label_knn[Validation_Label_size:] = 1

# 전체 학습용 데이터 라벨
Training_TotalLabel_knn = np.zeros(shape=DataNo*2)

Training_TotalLabel_knn[:DataNo] = 0
Training_TotalLabel_knn[DataNo:] = 1

Validation_Label_knn

## KNN(K-Nearest-Neighbor) 기반 모델링 및 성능평가

In [ ]:
Add    = 0
Divide = 0

for i in np.arange(Fold):
    c1 = 'Temp_train = Training_%d'%(i+1)
    exec(c1)
    c2 = 'Temp_val = Validation_%d'%(i+1)
    exec(c2)    

    knn = KNeighborsClassifier(n_neighbors = 3).fit(Temp_train,Training_Label_knn)
    
    c3 = 'knnscore_%d = knn.score(Temp_val, Validation_Label_knn)'%(i+1)
    exec(c3)
        
    Add += knn.score(Temp_val, Validation_Label_knn)
    Divide += 1
    
Avg_accuracy = Add/Divide

print(' Fold 1: {:.2f}% \n Fold 2: {:.2f}% \n Fold 3: {:.2f}% \n Fold 4: {:.2f}% \n Fold 5: {:.2f}%'.
        format(knnscore_1*100, knnscore_2*100, knnscore_3*100, knnscore_4*100, knnscore_5*100))
print('\n Average accuracy: {:.2f}%'.format(Avg_accuracy*100))

## KNN model 모델 생성(전체 데이터) 및 저장

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors = 2).fit(Training_TotalData,Training_TotalLabel_knn)

joblib.dump(KNN_model, './AI Model/KNN_model.plk')

## KNN model 불러와서 진단

In [ ]:
KNN_model_from_joblib = joblib.load('./AI Model/KNN_model.plk')
Prediction = KNN_model_from_joblib.predict(Validation_3)
Diagnosis_Result = pd.DataFrame(Prediction)
Diagnosis_Result
